###Fuente de la data

Los micro-datos gestionados en este notebook son tomados de www.datos.gov.co y corresponden a datos cuya fuente primaria es Departamento Nacional de Estadistica - Dane

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd

###Archivo DCD-area-proypoblacion-Mun-2020-2035-ActPostCOVID-19


- Contiene las proyecciones y retroproyecciones de población municipal para el periodo 2020-2035 con base en el CNPV 2018

In [55]:
# Leer el archivo desde path
path = '/content/drive/MyDrive/analytics_data_proyect/DCD-area-proypoblacion-Mun-2020-2035-ActPostCOVID-19.xlsx'
# Leer el archivo Excel, especificando que el encabezado está en la fila 9 (índice 8)
df_2020_2035 = pd.read_excel(path, sheet_name=0, header=8, dtype={'MPIO': str})

# Mostrar las primeras fila
df_2020_2035.head()

,DP,DPNOM,MPIO,DPMP,AÑO,ÁREA GEOGRÁFICA,Población
0,05,Antioquia,05001,Medellín,2020.0,Cabecera Municipal,2476569.0
1,05,Antioquia,05001,Medellín,2020.0,Centros Poblados y Rural Disperso,43023.0
2,05,Antioquia,05001,Medellín,2020.0,Total,2519592.0
3,05,Antioquia,05001,Medellín,2021.0,Cabecera Municipal,2506656.0
4,05,Antioquia,05001,Medellín,2021.0,Centros Poblados y Rural Disperso,42352.0


###Archivo DCD-area-proypoblacion-Mun-2005-2019.xlsx

Contiene las proyecciones y retroproyecciones de población municipal para el periodo 1985-2019

In [4]:
# Leer el archivo desde path
path = '/content/drive/MyDrive/analytics_data_proyect/DCD-area-proypoblacion-Mun-2005-2019.xlsx'
# Leer el archivo Excel, especificando que el encabezado está en la fila 9 (índice 8)
df_2005_2019 = pd.read_excel(path, sheet_name=0, header=11, dtype={'MPIO': str})

# Mostrar las primeras fila
df_2005_2019.head()

,DP,DPNOM,DPMP,MPIO,AÑO,ÁREA GEOGRÁFICA,Población
0,5,Antioquia,Medellín,05001,2005,Cabecera Municipal,1997763
1,5,Antioquia,Medellín,05001,2005,Centros Poblados y Rural Disperso,48578
2,5,Antioquia,Medellín,05001,2005,Total,2046341
3,5,Antioquia,Abejorral,05002,2005,Cabecera Municipal,7158
4,5,Antioquia,Abejorral,05002,2005,Centros Poblados y Rural Disperso,15784


##Revisión y limpieza para integrarlo a la base de datos

### - Resumen de la estructura del dataset

In [5]:
df_2020_2035.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53859 entries, 0 to 53858
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   DP               53859 non-null  object 
 1   DPNOM            53856 non-null  object 
 2   MPIO             53856 non-null  object 
 3   DPMP             53856 non-null  object 
 4   AÑO              53856 non-null  float64
 5   ÁREA GEOGRÁFICA  53856 non-null  object 
 6   Población        53856 non-null  float64
dtypes: float64(2), object(5)
memory usage: 2.9+ MB


In [6]:
df_2005_2019.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50490 entries, 0 to 50489
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   DP               50490 non-null  int64 
 1   DPNOM            50490 non-null  object
 2   DPMP             50490 non-null  object
 3   MPIO             50490 non-null  object
 4   AÑO              50490 non-null  int64 
 5   ÁREA GEOGRÁFICA  50490 non-null  object
 6   Población        50490 non-null  int64 
dtypes: int64(3), object(4)
memory usage: 2.7+ MB


- Suprimir las filas de total

In [8]:
df_filtered_2005 = df_2005_2019[df_2005_2019['ÁREA GEOGRÁFICA'] != 'Total']
df_filtered_2020 = df_2020_2035[df_2020_2035['ÁREA GEOGRÁFICA'] != 'Total']

- Ingtegrar verticalmente

In [9]:
# Verificar si tiene las mismas columnas
columns_2005 = set(df_filtered_2005.columns)
columns_2020 = set(df_filtered_2020.columns)

if columns_2005 == columns_2020:
    print("Ambos DataFrames tienen las mismas columnas.")
else:
    print("Los DataFrames tienen diferentes columnas.")
    print("Columnas en df_filtered_2005 pero no en df_filtered_2020:", columns_2005 - columns_2020)
    print("Columnas en df_filtered_2020 pero no en df_filtered_2005:", columns_2020 - columns_2005)

Ambos DataFrames tienen las mismas columnas.


In [10]:
# Concatenar los DataFrames verticalmente
df_combined = pd.concat([df_filtered_2005, df_filtered_2020], ignore_index=True)

- Peridos que abarca la informacion

In [13]:
df_combined['AÑO'].unique()

array([2005., 2006., 2007., 2008., 2009., 2010., 2011., 2012., 2013.,
       2014., 2015., 2016., 2017., 2018., 2019., 2020., 2021., 2022.,
       2023., 2024., 2025., 2026., 2027., 2028., 2029., 2030., 2031.,
       2032., 2033., 2034., 2035.,   nan])

### Eliminación de columnas irrelevantes para el proyecto

In [14]:
df_combined.head()

,DP,DPNOM,DPMP,MPIO,AÑO,ÁREA GEOGRÁFICA,Población
0,5,Antioquia,Medellín,05001,2005.0,Cabecera Municipal,1997763.0
1,5,Antioquia,Medellín,05001,2005.0,Centros Poblados y Rural Disperso,48578.0
2,5,Antioquia,Abejorral,05002,2005.0,Cabecera Municipal,7158.0
3,5,Antioquia,Abejorral,05002,2005.0,Centros Poblados y Rural Disperso,15784.0
4,5,Antioquia,Abriaquí,05004,2005.0,Cabecera Municipal,723.0


In [15]:
relevant_cols = ['DPNOM', 'MPIO', 'DPMP', 'AÑO', 'ÁREA GEOGRÁFICA', 'Población']
municipal_population_porjections_dane = df_combined[relevant_cols]
municipal_population_porjections_dane.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69567 entries, 0 to 69566
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   DPNOM            69564 non-null  object 
 1   MPIO             69564 non-null  object 
 2   DPMP             69564 non-null  object 
 3   AÑO              69564 non-null  float64
 4   ÁREA GEOGRÁFICA  69564 non-null  object 
 5   Población        69564 non-null  float64
dtypes: float64(2), object(4)
memory usage: 3.2+ MB


### Verificar valores nulos

In [16]:
municipal_population_porjections_dane.isnull().sum()

,0
DPNOM,3
MPIO,3
DPMP,3
AÑO,3
ÁREA GEOGRÁFICA,3
Población,3


- Ver filas con nulos

In [18]:
# Filtrar las filas que tienen al menos un valor nulo
filas_con_nulos = municipal_population_porjections_dane[municipal_population_porjections_dane.isnull().any(axis=1)]

# Mostrar las filas con valores nulos
print(filas_con_nulos)

      DPNOM MPIO DPMP  AÑO ÁREA GEOGRÁFICA  Población
69564   NaN  NaN  NaN  NaN             NaN        NaN
69565   NaN  NaN  NaN  NaN             NaN        NaN
69566   NaN  NaN  NaN  NaN             NaN        NaN


In [19]:
municipal_population_porjections_dane= municipal_population_porjections_dane.dropna()

In [20]:
municipal_population_porjections_dane.isnull().sum()

,0
DPNOM,0
MPIO,0
DPMP,0
AÑO,0
ÁREA GEOGRÁFICA,0
Población,0


### Estandarización de categorizaciones

La estandarización de categorizaciones es el proceso de uniformizar y normalizar los valores de las categorías en un conjunto de datos para asegurar la consistencia y evitar discrepancias. Esto es crucial para la calidad y precisión de los análisis

In [21]:
# Imprimir categorías únicas para columnas de tipo object
categorical_col = ['DPNOM', 'MPIO', 'DPMP', 'ÁREA GEOGRÁFICA']
for column in categorical_col:
    print(f"Categorías en la columna '{column}':")
    print(municipal_population_porjections_dane[column].unique())
    print()


Categorías en la columna 'DPNOM':
['Antioquia' 'Atlántico' 'Bogotá, D.C.' 'Bolívar' 'Boyacá' 'Caldas'
 'Caquetá' 'Cauca' 'Cesar' 'Córdoba' 'Cundinamarca' 'Chocó' 'Huila'
 'La Guajira' 'Magdalena' 'Meta' 'Nariño' 'Norte de Santander' 'Quindio'
 'Risaralda' 'Santander' 'Sucre' 'Tolima' 'Valle del Cauca' 'Arauca'
 'Casanare' 'Putumayo' 'Archipiélago de San Andrés' 'Amazonas' 'Guainía'
 'Guaviare' 'Vaupés' 'Vichada']

Categorías en la columna 'MPIO':
['05001' '05002' '05004' ... '99524' '99624' '99773']

Categorías en la columna 'DPMP':
['Medellín' 'Abejorral' 'Abriaquí' ... 'Cumaribo' 'Barrancominas'
 'Barrancominas*']

Categorías en la columna 'ÁREA GEOGRÁFICA':
['Cabecera Municipal' 'Centros Poblados y Rural Disperso']



- Borrar espacios en blanco al principio y al final, cambiar a mayúsculas, remover acentos y eliminar signos extraños

In [22]:
import unicodedata

def remove_accents_and_special_chars(input_str):
    # Normalizar la cadena a NFKD
    nfkd_form = unicodedata.normalize('NFKD', input_str)

    # Eliminar acentos
    no_accents = ''.join([c for c in nfkd_form if not unicodedata.combining(c)])

    # Definir caracteres no deseados
    unwanted_chars = [',', ';', '!', '?', '#', '$', '%', '"', "'", '/', '\\', '|', '-']

    # Eliminar caracteres no deseados
    cleaned_str = ''.join([c for c in no_accents if c not in unwanted_chars])

    # Remover espacios en blanco al principio y al final, y convertir a mayúsculas
    result = cleaned_str.strip().upper()

    return result

In [23]:
# Aplicar la función a todas las columnas categóricas
for col in categorical_col:
    municipal_population_porjections_dane.loc[:,col] = municipal_population_porjections_dane[col].apply(remove_accents_and_special_chars)

- Codificación de algunas variables categóricas

### Verificar que los valores en "MPIO" coincidan con los códigos reales de municipios

Los códigos reales de los municipios de Colombia, están almacenados en la base de datos PostgreSQL del proyecto, en la tabla municipalities dentro del campo dept_mpio_code, junto con la informacion necesaria para georeferenciar todos los municipios y departamentos de Colombia. Este campo guarda el código del municipio en un formato string de exactamente 5 caracteres.

Para hacer esta verificación, previamente exportamos desde la base de datos PostgreSQL un DataFrame con los siguientes campos: dept_name, mpio_name y dept_mpio_code, los cuales contienen la información de los departamentos y municipios oficiales, junto con sus respectivos códigos. Este DataFrame se carga en la siguiente celda y se utiliza para comparar con la columna "codigo_mpio"

In [25]:
dept_mpios_codes = pd.read_csv("/content/drive/MyDrive/analytics_data_proyect/deptos_mupios.csv", index_col=0, dtype={'dept_mpio_code': str})
print(dept_mpios_codes.info())
dept_mpios_codes.head()

<class 'pandas.core.frame.DataFrame'>
Index: 1121 entries, 0 to 1120
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   dept_mpio_code  1121 non-null   object
 1   dept_name       1121 non-null   object
 2   mupio_name      1121 non-null   object
dtypes: object(3)
memory usage: 35.0+ KB
None


,dept_mpio_code,dept_name,mupio_name
0,97001,VAUPES,MITU
1,97161,VAUPES,CARURU
2,97511,VAUPES,PACOA
3,97666,VAUPES,TARAIRA
4,97777,VAUPES,PAPUNAHUA


 -  Verificar la consistencia de la columna "MPIO" en el df municipal_population_porjections_dane

In [26]:
# Asegurarnos de que todos los valores en 'codigo_m' sean strings
municipal_population_porjections_dane.loc[:,'MPIO'] = municipal_population_porjections_dane['MPIO'].astype(str)

# Calcular la longitud de cada valor en la columna
longitudes = municipal_population_porjections_dane['MPIO'].apply(len)

# Verificar si todas las longitudes son iguales
longitudes.nunique() == 1

True

In [27]:
# Mostrar longitudes únicas (opcional)
print(f"Longitudes únicas: {longitudes.unique()}")

Longitudes únicas: [5]


In [28]:
# Contar registros por longitud
long_df = longitudes.value_counts().reset_index()
long_df.rename(columns={'codigo_dane': 'no_dígitos_codigo_dane'}, inplace=True)
long_df['percentage'] = (long_df['count'] / len(longitudes))
long_df.head()

,MPIO,count,percentage
0,5,69564,1.0


In [29]:
# Mostrar una muestra de registros para cada longitud
for longitud in longitudes.value_counts().index:
    print(f"Muestra de registros con longitud {longitud}:")
    muestra = municipal_population_porjections_dane[longitudes == longitud].head(5)  # Muestra de los primeros 5 registros
    print(muestra[['MPIO']])
    print()

Muestra de registros con longitud 5:
    MPIO
0  05001
1  05001
2  05002
3  05002
4  05004



Nota: los cod de los municipios del df tiene la misma estructura de los que se guardan en la base de datos: string de 5 digitos

- Verificar que los códigos de municipios que quedaron en el dataset correspondan solamente a códigos reales

In [30]:
# Función para comparar listas y mostrar diferencias
def compare_lists(df1_col, df2_col, label1, label2):
    # Extraer listas únicas y normalizar
    list1 = set(df1_col.str.strip().str.upper().unique())
    list2 = set(df2_col.str.strip().str.upper().unique())

    # Encontrar diferencias
    only_in_list1 = list1 - list2
    only_in_list2 = list2 - list1

    # Imprimir resultados
    print(f"{label1} que no están en {label2}:")
    print(only_in_list1)

In [31]:
# Comparar listas de códigos
compare_lists(municipal_population_porjections_dane['MPIO'], dept_mpios_codes['dept_mpio_code'],
              "Códigos de municipios en municipal_population_porjections_dane", "Códigos de municipios dept_mpios_codes")

Códigos de municipios en municipal_population_porjections_dane que no están en Códigos de municipios dept_mpios_codes:
{'94663'}


  - Identifcar los registros bajo este código que no corresponde a ningun código real de municipio

In [33]:
cod_wront = municipal_population_porjections_dane[municipal_population_porjections_dane['MPIO'] == '94663']
cod_wront['DPMP'].unique()

array(['MAPIRIPANA (ANM)'], dtype=object)

Mapiripana una poblacion del Guania que en la tabla geolocalizada de municipios de la bd figura como corregimiento del municipio BarrancoMinas de Guania. Por lo tanto reemplazamos este codigo por el de Municipio al que pertenece

In [34]:
# Listado de municipios de Guania
dept_mpios_codes[dept_mpios_codes['dept_name'] == 'GUAINIA']

,dept_mpio_code,dept_name,mupio_name
1092,94001,GUAINIA,INIRIDA
1093,94343,GUAINIA,BARRANCOMINAS
1094,94883,GUAINIA,SAN FELIPE
1095,94884,GUAINIA,PUERTO COLOMBIA
1096,94885,GUAINIA,LA GUADALUPE
1097,94886,GUAINIA,CACAHUAL
1098,94887,GUAINIA,PANA PANA
1099,94888,GUAINIA,MORICHAL


In [35]:
municipal_population_porjections_dane.loc[:,'MPIO'] = municipal_population_porjections_dane['MPIO'].replace( '94663', '94343')

- Verificar nuevamente que los códigos de municipios que quedaron en el dataset correspondan solamente a códigos reales

In [36]:
# Comparar listas de códigos
compare_lists(municipal_population_porjections_dane['MPIO'], dept_mpios_codes['dept_mpio_code'],
              "Códigos de municipios en municipal_population_porjections_dane", "Códigos de municipios dept_mpios_codes")

Códigos de municipios en municipal_population_porjections_dane que no están en Códigos de municipios dept_mpios_codes:
set()


### Procesamiento final como preparación para integrarlo a la bd de datos del proyecto

In [38]:
# Eliminar columnas que ya no se necesitan
columns_to_drop = ['DPNOM', 'DPMP']
municipal_population_porjections_dane = municipal_population_porjections_dane.drop(columns=columns_to_drop)

In [40]:
# Adicionar columna para trazabilidad de la fuente
municipal_population_porjections_dane['SOURCE_ID'] = municipal_population_porjections_dane['AÑO'].apply(
    lambda x: 127 if 2005 <= x <= 2019 else
              128 if 2020 <= x <= 2035 else None
)

In [44]:
municipal_population_porjections_dane.columns

Index(['MPIO', 'AÑO', 'ÁREA GEOGRÁFICA', 'Población', 'SOURCE_ID'], dtype='object')

In [46]:
# Ajustar nombre de columnas

# Definir el diccionario de traducción
translation_map = {
    'MPIO': 'dane_code',
    'AÑO': 'year',
    'ÁREA GEOGRÁFICA': 'geographic_area',
    'Población': 'population',
    'SOURCE_ID': 'source_id'
}

# Renombrar las columnas
municipal_population_porjections_dane.rename(columns=translation_map, inplace=True)

In [47]:
#Estructura final del dataset a integrar a la base de datos
municipal_population_porjections_dane.info()

<class 'pandas.core.frame.DataFrame'>
Index: 69564 entries, 0 to 69563
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   dane_code        69564 non-null  object 
 1   year             69564 non-null  float64
 2   geographic_area  69564 non-null  object 
 3   population       69564 non-null  float64
 4   source_id        69564 non-null  int64  
dtypes: float64(2), int64(1), object(2)
memory usage: 3.2+ MB


In [50]:
# Convertir la columna 'year' de float a int
municipal_population_porjections_dane['year'] = municipal_population_porjections_dane['year'].astype(int)

In [51]:
municipal_population_porjections_dane.head()

,dane_code,year,geographic_area,population,source_id
0,05001,2005,CABECERA MUNICIPAL,1997763.0,127
1,05001,2005,CENTROS POBLADOS Y RURAL DISPERSO,48578.0,127
2,05002,2005,CABECERA MUNICIPAL,7158.0,127
3,05002,2005,CENTROS POBLADOS Y RURAL DISPERSO,15784.0,127
4,05004,2005,CABECERA MUNICIPAL,723.0,127


## Salvar en archivo csv en el drive

In [54]:
# Guardar en archivos CSV en el drive
municipal_population_porjections_dane.to_csv('/content/drive/MyDrive/analytics_data_proyect/initial_transformation/municipal_population_porjections_dane.csv', index=False)